# Yt 2D Slice Plots Cookbook
## Kian Hayes

In this notebook, I'll be going over some of the tecniques I've used to produce Slice Plots in Yt. We'll first go over simple 2D slice plots and then move onto some more advanced things you can build on top of that to get some unique plots that will help extract different kinds of information. 

# Basic Slice Plot

Getting a Slice plot is probably one of the most simple things you can do in YT to easily visualize your data without much computation power. We do the following:

In [ ]:
import yt

ds = yt.load('/gpfs/projects/CalderGroup/BrendanSpace/SIM_DATA/hires_urca_problem/plt0030084')
field = ('boxlib', 'X(na23)')

slc = yt.SlicePlot(ds, 'z', field)
slc.show()

First we import yt, load our data that we want to look at, and specify our field we want to look at within the data. The syntax for plotting your data to a SlicePlot is storing the `yt.SlicePlot` object in a variable. This take the arguments `(data, direction, field)` where `data` is our dataset, `direction` is the Cartesian coordinate direction that YT will slice our data, and `field` is the field we want to look at. This can be a tuple of strings as is the format in most simulation plot files but we have this stored in a variable so the argument will often share the name of our variable. Finally, we call YT to show our Slice Plot and it will output it in the output of our Notebook. If we want to save the image rather, we would use `slc.save(PATH)` where `PATH` is a specified directory. 

# Time Series 2D Slice Plots

We often might want to see our simulation over time. In this case we want to output a SlicePlot of each time step we have. The key to this is how we load our data into the `ds` variable. In YT, if we use a wildcard inside of the `yt.load`, it will store all plot files in a series in which we can loop over in a for loop. With that, plotting for each time step is a simple for loop:

In [ ]:
import yt

ts = yt.load('/gpfs/projects/CalderGroup/KianSpace/snia_hddt_runs/june-25-results/hdef_hdf5_plt_cnt*')
field = ('gas', 'temp')

for ds in ts:
    slc = yt.SlicePlot(ds, 'z', field)
    slc.show()

# Slice Plot for every Direction (3D Data)

If our data is in 3 dimensions, we have a choice for what axes to show on our 2 dimensional plot! With this, we can look at our data at different perspectives which may show different phenemona depending on the axes used. If we know what coordinate system the simulation uses then we can simply loop through a list of these directions and change the direction argument in our Slice Plot. For this example, our data is in Cartestian coordinates (they could also be in Cylindircal, or Spherical)

In [ ]:
import yt

ts = yt.load('/gpfs/projects/CalderGroup/KianSpace/snia_hddt_runs/june-25-results/hdef_hdf5_plt_cnt*')
field = ('gas', 'temp')

directions = ['x', 'y', 'z']

for direction in directions:
    slc = yt.SlicePlot(ds, direction, field)
    slc.show()

# Rotating around an axis

We may want to spin our data around an axis to get a look into the whole 3 dimensional space. For that, dust off a bit of your trigonometry because rotating the 'camera' around our source is not as trivial as it was in the 3D render. With that, our naming convention for saving each image is also a little involved. To do this we'll use a numpy linspace array to store every angle at which we rotate around the source that we want an image at. Set up/load like regular:

In [ ]:
import yt
import os
import numpy as np

yt.enable_parallelism()

ds = yt.load('/gpfs/projects/CalderGroup/BrendanSpace/SIM_DATA/hires_urca_problem/plt0030084')

field = ('boxlib', 'X(na23)')
thetas = np.linspace(0, 2*np.pi, 60)
normal_direction = 'y'

Here we enable parallelism but I've noticed that it hardly makes a difference when making Slice Plots. `np.linspace` takes the arguments `(start, stop, step)` and is inclusive. In our case, we start at an angle of 0 and go all the way around to 2pi which will be in 60 steps. This means that we will have 60 images when our loop is done. (NOTE: numpy works best in radians. We'll convert to degrees later though as it's a bit more intuitive for titling our graph). Next, we've defined the cartestian plane that we want our slice to be or the vector that will be normal from the center of our data to the 'camera' we're viewing it through (There isn't really a camera but for intuition we can think of it like that). In this case we've chosen the x plane. This means that the axis to our plot will be z and y. This is important to keep in mind when we're deciding how to change the angle. Now we want to loop through this array to have it change the angle we're rotate around. The first part of this for loop is associated with the file name:

In [ ]:
for theta in thetas:

    # Setting up file name
    for i in range(len(thetas)):
        if theta == thetas[i]:
            filename = f'{normal_direction}_{field[1]}_rot{i+1}.png'
        else:
            continue
        
    normal_vector = round(np.cos(theta), 2)
    secondary_vector = round(np.sin(theta), 2)
    degrees = theta * (180/np.pi)
    
    slc = yt.OffAxisSlicePlot(ds, 
                              normal=(0, normal_vector, secondary_vector), 
                              fields=field, north_vector=(1,0,0), 
                              center=ds.domain_center, 
                              width=(1000, 'km'))
    
    if yt.is_root():
        slc.save(f'/gpfs/projects/CalderGroup/KianSpace/plots/urca/slices/rotate/ye/images/{filename}')

I've mentioned previously the importance of having unique names for our images we want to string together given that ffmpeg is easiest to use when it can follow a pattern such as an increment of integer with our file names. We have a nested for loop to check the angle we are currently on against the index of that angle in our angle array. This will ensure that the image associated with the angle 0 will have '1' in its file name and when we're at an angle of 2pi it has '60' in the file name. YT doesn't know how to do this so we have to do it ourselves. This may not be the most efficient or quickest way to do this, but when we're looping through an array that's only 60 elements it doesn't make too much a difference as far as computation time. 

Our data is in Cartesian coordinates. This means that angles do not play well unless we calculate the vector compenents associated with our angle. This is where trig comes in. How I imagined the geometry to calculate this correctly is a bit tricky to explain in words precisely without a picture. All that you should keep in mind is that the normal vector that we've chosen is associated with the cosine of our angle and the direction on the x-axis of our plot which I've called the "secondary vector" is associated with the sine of our angle. In this step we've also calculated our conversion to degrees from radians. 

From this, it's a matter of calling YT to make the Slice plot with our calculated values. Note that we use `OffAxisSlicePlot` rather since we are not longer orthogonal to our regular coordinate system. Try not to use `SlicePlot` because Yt may give the file names where angle is orthogonal to the coordinate system something that is inconsistant with the pattern we have set. We have a specific file name set already so this may not matter but to be sure I wouldn't use `SlicePlot`.

The two important arguments in this are `normal_vector` and `north_vector` which take the sequence of floats (x, y, z). `normal_vector` is essentially the vector in the direction from the center of our data to the camera we're viewing the data through. The `north_vector` should point directly up from the data with respect to the plane that we're viewing through. Since our normal vector will never have magnitude in the direction of our north vector, we set the direction that we're not using (AKA the north vector) in this sequence to zero. In our case, the normal vector starts on the y axis then rotates around the x axis (X is our north vector). So we need to change y and z in our normal vector to our calculated values and keep the x component in the sequence as 0. We'll set the argument `north_vector` to align to the x axis to ensure that the north vector stays consistant. 

Sometime you may make a mistake and notice that in your sequence of images, you see the image flip for some reason instead of smooth rotation. You likely messed up the above geometry in some way if you see this so go back and think carefully about how you aligned the vectors. 

# Putting images together with FFMPEG

Stringing our images together is largely the same as when we did it with the 3D renders so this section will basically be the same as that section from the 3D render notebook. 

In a Seawulf terminal, you'll want to import the ffmpeg module with:

```
module import ffmpeg
```

cd into where you have all the images you want to stitch. From the previous example, our images follow the pattern 'y_X(na23)_rot1.png' with the integer after 'rot' changing incrementally. I've signified what field we're looking at, and what number in the iteration we are in so they're in order of how the camera rotated around the source. With that, we can then type in the command to string them together:

```
ffmpeg -r 10 -stream_loop 3 -i '3d_render_X(na23)_rot%1d.png' movie.mp4
```

-r sets the frame rate of our video, -i sets the pictures we want to string together, and -stream_loop sets the amount of time to loop the video, 3 times in this case. Specifically within this string, we have a pattern matching syntax of '%1d'. Basically this means that the picture names we want are in order starting at 1 to however many pictures we have. There are other ways to pattern match using this method but I haven't looked into it that extensively because it's a confusing system. Finally, we need an output name which is 'movie.mp4' in this case but can be anything of course. 


# Useful Parameters to change

There are a number of parameters that we can change about our SlicePlot. I've dug through the docs to find these parameters and have compiled the most useful ones (in my opinion) here.

### Change Color map

In [ ]:
slc.set_cmap(field, 'twilight_tshifted')

Changes the color map. To get a list of color maps, look up 'matplotlib color maps'.

### Set log space or linear space

In [ ]:
slc.set_log(field, False)

Changes the color map axis to be either in linear space of log space. This can drastically change how your plot looks. `False` changes to linear space, `True` sets to log space. I would only use linear space if your values change over orders of magnitudes but you can mess with it to see how it effects the coloring on your plot (it could turn out something cool!).

### Set the color bar bounds

In [ ]:
slc.set_zlim(field, zmin=3e-5, zmax=3.35e-5)

Changes the bounds of your color map. I play with this one a lot to capture certain values. 

### Add contour line

In [ ]:
slc.annotate_contour(('boxlib', 'rho'), levels=1, factor=1, clim=(1.9e9, 1.9e9))

Adds a contour annotation to your plot. This was useful when trying to denote where the Urca Shell was on the plot.

### Change color bar label

In [ ]:
slc.set_colorbar_label(field, '')

Changes the side bar label. I have it set to be blank in this case.

### Hide x and y axes

In [ ]:
slc.hide_axes()

### Hide color bar ticks

In [ ]:
slc.set_colorbar_minorticks(field, False)

Sets whether the minor ticks show on the color bar. In this case, this setting is off.

### Change plot title

In [ ]:
slc.annotate_title(f'Angle Rotated Around Source (degrees): {round(degrees, 2)}')

### Change font size

In [ ]:
slc.set_font_size(18)

Changes the font size of every piece of text including the axes values, axes names, colorbar name, colorbar values, and plot title. 